#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [5]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                150000 non-null  int64  
 1   name              149998 non-null  object 
 2   category          150000 non-null  object 
 3   main_category     150000 non-null  object 
 4   currency          150000 non-null  object 
 5   deadline          150000 non-null  object 
 6   goal              150000 non-null  object 
 7   launched          150000 non-null  object 
 8   pledged           150000 non-null  object 
 9   state             150000 non-null  object 
 10  backers           150000 non-null  object 
 11  country           150000 non-null  object 
 12  usd pledged       148518 non-null  object 
 13  usd_pledged_real  150000 non-null  float64
dtypes: float64(1), int64(1), object(12)
memory usage: 16.0+ MB


In [6]:
df_ks.isnull().sum()

ID                     0
name                   2
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         1482
usd_pledged_real       0
dtype: int64

### Importer les données

In [7]:
dict_donnees = df_ks.to_dict('records')
collection.insert_many(dict_donnees)

## Question 1  

In [9]:
collection.find_one(sort=[("usd_pledged_real", pymongo.DESCENDING)])

{'_id': ObjectId('5fd761ed0b30c1cc51ff9e04'),
 'ID': 342886736,
 'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
 'category': 'Product Design',
 'main_category': 'Design',
 'currency': 'USD',
 'deadline': '2014-08-30',
 'goal': '50000.0',
 'launched': '2014-07-08 10:14:37',
 'pledged': '13285226.36',
 'state': 'successful',
 'backers': '62642',
 'country': 'US',
 'usd pledged': '13285226.36',
 'usd_pledged_real': 13285226.36}

In [11]:
cur = collection.find({}).sort([("usd_pledged_real", pymongo.DESCENDING)]).limit(5)
list(cur)

[{'_id': ObjectId('5fd761ed0b30c1cc51ff9e04'),
  'ID': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2014-08-30',
  'goal': '50000.0',
  'launched': '2014-07-08 10:14:37',
  'pledged': '13285226.36',
  'state': 'successful',
  'backers': '62642',
  'country': 'US',
  'usd pledged': '13285226.36',
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('5fd761ed0b30c1cc51fff8fd'),
  'ID': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-30',
  'goal': '1000000.0',
  'launched': '2016-05-24 15:49:52',
  'pledged': '12779843.49',
  'state': 'successful',
  'backers': '66673',
  'country': 'US',
  'usd pledged': '12779843.49',
  'usd_pledged_real': 12779843.49},
 {'_id': ObjectId('5fd761ed0b30c1cc51002e18'),
  'ID': 1033978702,
  'name': 'OUY

## Question 2

In [12]:
collection.find({"state" : "successful"}).count(True)

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


53040

## Question 3

In [14]:
categories = collection.distinct("category")
categories

['3D Printing',
 'Academic',
 'Accessories',
 'Action',
 'Animals',
 'Animation',
 'Anthologies',
 'Apparel',
 'Apps',
 'Architecture',
 'Art',
 'Art Books',
 'Audio',
 'Bacon',
 'Blues',
 'Calendars',
 'Camera Equipment',
 'Candles',
 'Ceramics',
 "Children's Books",
 'Childrenswear',
 'Chiptune',
 'Civic Design',
 'Classical Music',
 'Comedy',
 'Comic Books',
 'Comics',
 'Community Gardens',
 'Conceptual Art',
 'Cookbooks',
 'Country & Folk',
 'Couture',
 'Crafts',
 'Crochet',
 'DIY',
 'DIY Electronics',
 'Dance',
 'Design',
 'Digital Art',
 'Documentary',
 'Drama',
 'Drinks',
 'Electronic Music',
 'Embroidery',
 'Events',
 'Experimental',
 'Fabrication Tools',
 'Faith',
 'Family',
 'Fantasy',
 "Farmer's Markets",
 'Farms',
 'Fashion',
 'Festivals',
 'Fiction',
 'Film & Video',
 'Fine Art',
 'Flight',
 'Food',
 'Food Trucks',
 'Footwear',
 'Gadgets',
 'Games',
 'Gaming Hardware',
 'Glass',
 'Grace is Leaving',
 'Graphic Design',
 'Graphic Novels',
 'Hardware',
 'Hip-Hop',
 'Horror',


In [15]:
for i in categories :
    print(i, " : ", collection.find( { "category": i } ).count())

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


3D Printing  :  271
Academic  :  367
Accessories  :  1204
Action  :  282
Animals  :  97
Animation  :  1017
Anthologies  :  300
Apparel  :  2827
Apps  :  2535
Architecture  :  300
Art  :  3358
Art Books  :  1065
Audio  :  164
Bacon  :  78
Blues  :  113
Calendars  :  112
Camera Equipment  :  165
Candles  :  168
Ceramics  :  128
Children's Books  :  2686
Childrenswear  :  192
Chiptune  :  10
Civic Design  :  130
Classical Music  :  1064
Comedy  :  923
Comic Books  :  1032
Comics  :  1931
Community Gardens  :  115
Conceptual Art  :  393
Cookbooks  :  217
Country & Folk  :  1790
Couture  :  108
Crafts  :  1834
Crochet  :  61
DIY  :  480
DIY Electronics  :  358
Dance  :  901
Design  :  1641
Digital Art  :  524
Documentary  :  6498
Drama  :  871
Drinks  :  995
Electronic Music  :  858
Embroidery  :  49
Events  :  322
Experimental  :  357
Fabrication Tools  :  96
Faith  :  439
Family  :  130
Fantasy  :  132
Farmer's Markets  :  175
Farms  :  482
Fashion  :  3379
Festivals  :  313
Fiction  :  3

## Question 4

In [17]:
from datetime import datetime

cur = collection.find({"country":"FR", "launched": {"$lt" : datetime(2016, 1, 1, 0, 0, 0)}})
list(cur)

[]

## Question 5

In [19]:
cur = collection.find({ "goal" : {"$gte":2000}})
list(cur)

[{'_id': ObjectId('5fd761ec0b30c1cc51fe72df'),
  'ID': 872782264,
  'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
  'category': 'Rock',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2011-09-16',
  'goal': 2000.0,
  'launched': '2011-08-17 06:31:31',
  'pledged': 1145.0,
  'state': 'canceled',
  'backers': 24,
  'country': 'US',
  'usd pledged': 1145.0,
  'usd_pledged_real': 1145.0},
 {'_id': ObjectId('5fd761ec0b30c1cc51fe72e0'),
  'ID': 1326492673,
  'name': 'Ohceola jewelry',
  'category': 'Fashion',
  'main_category': 'Fashion',
  'currency': 'USD',
  'deadline': '2012-08-22',
  'goal': 18000.0,
  'launched': '2012-07-23 20:46:48',
  'pledged': 1851.0,
  'state': 'failed',
  'backers': 28,
  'country': 'US',
  'usd pledged': 1851.0,
  'usd_pledged_real': 1851.0},
 {'_id': ObjectId('5fd761ec0b30c1cc51fe72e1'),
  'ID': 1688410639,
  'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"',
  'category': 'Tabletop Games',
  'main_category': 

## Question 6 

In [20]:
cur=collection.find({"name" : {'$regex' : "Sport" }})
len(list(cur))

323